# Локальный поиск в графе - Graph Local Search

In [1]:
import pandas as pd
import os
import tiktoken

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.config.enums import ModelType
from graphrag.config.models.language_model_config import LanguageModelConfig
from graphrag.language_model.manager import ModelManager
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

In [4]:
GRAPHRAG_DIR = os.path.join(os.getcwd(), "ragtestour")
INPUT_DIR = os.path.join(GRAPHRAG_DIR, "output")
LANCEDB_URI = os.path.join(INPUT_DIR, "lancedb") 

In [5]:
COMMUNITY_REPORT_TABLE = "community_reports"
COMMUNITY_TABLE = "communities"
ENTITY_TABLE = "entities"
RELATIONSHIP_TABLE = "relationships"
COVARIATE_TABLE = "covariates"
TEXT_UNIT_TABLE = "text_units"
COMMUNITY_LEVEL = 2

In [6]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
community_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")

entities = read_indexer_entities(entity_df, community_df, COMMUNITY_LEVEL)

relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, community_df, COMMUNITY_LEVEL)

text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

In [7]:
description_embedding_store = LanceDBVectorStore(
    collection_name="default-entity-description",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)

full_content_embedding_store = LanceDBVectorStore(
    collection_name="default-community-full_content",
)
full_content_embedding_store.connect(db_uri=LANCEDB_URI)

In [8]:
# api_key_llm = "not-needed"
api_key = os.environ["GRAPHRAG_API_KEY"]
# api_key

llm_model = "gpt-4o-mini" # "gemma-3-27b-it@q4_k_m"
embedding_model = "text-embedding-3-small"

In [9]:
chat_config = LanguageModelConfig(
    api_key=api_key,
    api_base="https://api.vsegpt.ru/v1", # "http://172.21.16.141:1234/v1",
    type=ModelType.OpenAIChat,
    model=llm_model,
    max_retries=20,
)
chat_model = ModelManager().get_or_create_chat_model(
    name="local_search",
    model_type=ModelType.OpenAIChat,
    config=chat_config,
)

token_encoder = tiktoken.encoding_for_model(llm_model)

embedding_config = LanguageModelConfig(
    api_key=api_key,
    api_base="https://api.vsegpt.ru/v1",
    type=ModelType.OpenAIEmbedding,
    model=embedding_model,
    max_retries=20,
)

text_embedder = ModelManager().get_or_create_embedding_model(
    name="local_search_embedding",
    model_type=ModelType.OpenAIEmbedding,
    config=embedding_config,
)

In [10]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # if you did not run covariates during indexing, set this to None
    covariates=None, #covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

In [11]:
local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

model_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [12]:
search_engine = LocalSearch(
    model=chat_model,
    context_builder=context_builder,
    token_encoder=token_encoder,
    model_params=model_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [13]:
import time

In [14]:
start = time.time()

result = await search_engine.search("Что такое межрасчётные выплаты?")

end = time.time()
print(f"Длительность: {(end - start):.3f} сек.")

Длительность: 12.991 сек.


In [15]:
print(result.response)

## Определение межрасчётных выплат

Межрасчётные выплаты представляют собой финансовые средства, которые выплачиваются работникам вне регулярного расчётного периода. Эти выплаты могут включать различные виды компенсаций, такие как выплаты за неиспользованные дни отпуска, дополнительные выплаты в связи с изменениями в трудовых условиях или другие разовые начисления, которые не входят в стандартный расчёт заработной платы [Data: Entities (276); Relationships (1324, 320)].

## Процессы, связанные с межрасчётными выплатами

Межрасчётные выплаты инициируются на основании различных документов, таких как приказ о предоставлении отпуска. После подписания такого приказа начинается процесс формирования межрасчётных выплат, что подчеркивает важность соблюдения установленных процедур в управлении трудовыми ресурсами [Data: Relationships (1302, 955, 581)].

Кроме того, межрасчётные выплаты могут быть частью расчёта авансов, что позволяет обеспечить дополнительные выплаты между основными расчётами. 

In [21]:
result.context_data.keys()

dict_keys(['reports', 'relationships', 'entities', 'sources'])

In [20]:
result.context_data['entities']

dict_keys(['reports', 'relationships', 'entities', 'sources'])

In [19]:
result.context_data['relationships']

,id,source,target,description,weight,links,in_context
0,548,ПЕРЕРАСЧЕТ ЗАРПЛАТЫ,ОРГАНИЗАЦИЯ ПЕРЕРАСЧЕТОВ,Организация перерасчетов связана с процессом п...,1.0,1,True
1,1324,МЕЖРАСЧЕТНЫЕ ВЫПЛАТЫ,ЗАРПЛАТА,Межрасчетные выплаты представляют собой дополн...,1.0,4,True
2,320,ЗАРПЛАТА,МЕЖРАСЧЕТНЫЕ ВЫПЛАТЫ,Межрасчетные выплаты отражаются в системе и яв...,8.0,4,True
3,329,РАЗОВЫЕ НАЧИСЛЕНИЯ,ЗАРПЛАТА,Разовые начисления влияют на расчет заработной...,8.0,4,True
4,321,ЗАРПЛАТА,РАЗОВЫЕ НАЧИСЛЕНИЯ,Разовые начисления и удержания влияют на общую...,7.0,4,True
5,1321,РЕГУЛЯРНЫЙ РАСЧЕТ ЗАРПЛАТЫ,ЗАРПЛАТА,Регулярный расчет заработной платы включает в ...,9.0,4,True
6,316,ЗАРПЛАТА,ИЗМЕНЕНИЯ В СПОСОБАХ ВЫПЛАТ,Изменения в способах выплат влияют на процесс ...,1.0,4,True
7,2550,ЭТЖ,ПРЕМИИ И НАДБАВКИ,Процесс ЭТЖ включает заполнение области премий...,8.0,3,True
8,1073,ЭТЖ,ПЕРЕРАСЧЕТ ЗАРПЛАТЫ,Процесс перерасчета заработной платы включает ...,1.0,3,True
9,330,РАЗОВЫЕ НАЧИСЛЕНИЯ,ЭТЖ,Процесс расчета заработной платы включает разо...,8.0,3,True


In [27]:
print(result.context_data['reports'].content[0])

# Human Resources Software Module Documentation Analysis

The community focuses on the documentation processes within the human resources software module, particularly concerning payroll, employee onboarding, and compliance with labor regulations. Key entities include payroll calculations, deductions, and the roles of various stakeholders such as accountants and HR specialists, all of which interact to ensure accurate and timely payroll processing.

## Impact of Deductions on Payroll

Deductions play a crucial role in determining the final payroll amount, as they reduce the gross salary of employees. This relationship highlights the importance of accurate deduction calculations to ensure employees receive the correct net pay. The integration of deductions into the payroll process is essential for compliance with financial obligations and employee satisfaction. [Data: Entities (271, 41); Relationships (331)]

## Role of Payroll Accountants

Payroll accountants, specifically those in the